# BentoML Example: fashion-mnist with tf-keras

# !IMPORTANT - this notebook is still working-in-progress, not everything works

[BentoML](http://bentoml.ai) is an open source framework for building, shipping and running machine learning services. It provides high-level APIs for defining an ML service and packaging its artifacts, source code, dependencies, and configurations into a production-system-friendly format that is ready for deployment.

This notebook demonstrates how to use BentoML to turn a tf-keras model into a docker image containing a REST API server serving this model, how to use your ML service built with BentoML as a CLI tool, and how to distribute it a pypi package.

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=&t=event&ec=nb&ea=open&dp=tf-keras-fashion-mnist&dt=tf-keras-fashion-mnist)

In [ ]:
!pip install -I bentoml
!pip install tensorflow numpy matplotlib

In [ ]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

import bentoml

## Import the Fashion MNIST dataset

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
# Specify the class names explicitly for later use
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Preprocessing the data

The data must be preprocessed before training the network. If you inspect the first image in the training set, you will see that the pixel values fall in the range of 0 to 255:

In [ ]:
plt.figure()
plt.imshow(train_images[0], cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

We should scale these values to a range of 0 to 1 before feeding to the neural network model. For this, we divide the values by 255. It's important that the *training set* and the *testing set* are preprocessed in the same way:

In [ ]:
train_images = train_images / 255.0

test_images = test_images / 255.0

## Displaying 25 images from the training and testing sets

In [ ]:
def show_25(images, labels):
    
    plt.figure(figsize=(10,10))
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)
        plt.xlabel(class_names[labels[i]])
    plt.show()

In [ ]:
# From the train set
show_25(train_images, train_labels)

In [ ]:
# From the test set
show_25(test_images, test_labels)

## Building a simple ConvNet

In [ ]:
# Reshaping the images so that our model can accept it
train_images  = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

In [ ]:
# Define the Convolutional Neural Network
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu')
])

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

## Compiling the model

In [ ]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Training the model

In [ ]:
model.fit(train_images, train_labels, epochs=5)

## Evaluation on the test set

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)

## Making prediction on a single image

Let's first see the test image and and its label:

In [ ]:
plt.imshow(test_images[0].squeeze(-1), cmap=plt.cm.binary)
plt.show()
print('\nClass', test_labels[0])

In [ ]:
import numpy as np

model.predict_classes(np.expand_dims(test_images[0], 0))

# Define ML service with BentoML

In [ ]:
%%writefile tf_keras_fashion_mnist.py

import bentoml
from bentoml import api, artifacts, env, BentoService
from bentoml.artifact import TfKerasModelArtifact, PickleArtifact
from bentoml.handlers import ImageHandler
from scipy.misc import imresize
import numpy as np


@bentoml.env(conda_dependencies=['tensorflow', 'numpy'])
@bentoml.artifacts([TfKerasModelArtifact('classifier'), PickleArtifact('class_names')])
class TfKerasFashionMnistModel(bentoml.BentoService):
    
    @bentoml.api(ImageHandler, pilmode='L')
    def predict(self, image_array):
        image_array_data = imresize(image_array, (28, 28))
        image_array_data = image_array_data.reshape(1, 28, 28, 1)
        
        # Make prediction with our saved model
        result = self.artifacts.classifier.predict(image_array_data)
        
        # 
        category_int = np.argmax(result, axis=1)
        return self.artifacts.class_names[int(category_int)]

# Save BentoML service archive

In [ ]:
from tf_keras_fashion_mnist import TfKerasFashionMnistModel

bento_model = TfKerasFashionMnistModel.pack(classifier=model, class_names=class_names)
saved_path = bento_model.save('/tmp/bento')
print(saved_path)

# Load from BentoML service archive

In [ ]:
img = test_images[0]
svc = bentoml.load(saved_path)
svc.predict(img)

*For demo purpurse, copy generated model to ./model folder

In [ ]:
import os
import shutil
shutil.rmtree('./model', ignore_errors=True)
shutil.copytree(saved_path, './model')

# "pip install" a BentoML archive

BentoML user can directly pip install saved BentoML archive with `pip install $SAVED_PATH`,  and use it as a regular python package.

In [ ]:
!pip install ./model

In [ ]:
# Your bentoML model class name will become packaged name
import TfKerasFashionMnistModel

ms = TfKerasFashionMnistModel.load()
ms.predict(img)

# Run REST API server with Docker

** _Note: `docker` is not available when running in Google Colaboratory_

In [ ]:
!cd "./model" && docker build -t tf-keras-fashion-mnist .

In [ ]:
!docker run -p 5000:5000 tf-keras-fashion-mnist